<a href="https://colab.research.google.com/github/unique-subedi/gene-expression/blob/main/Xinhe_gene_expression_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import os
import datetime
import time
import math
import numpy as np
from numpy import linalg as LA
import pandas as pd
import urllib.request
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multitest import multipletests

In [2]:
!pip install pyreadr
import pyreadr
urllib.request.urlretrieve("https://raw.githubusercontent.com/unique-subedi/gene-expression/main/data/brain.rda", "brain.rda")
brain = pyreadr.read_r("brain.rda")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 363 kB 4.8 MB/s 


In [3]:
expression = pd.DataFrame(brain["expression"])
genes = pd.DataFrame(brain["genes"])
samples = pd.DataFrame(brain["samples"])

In [5]:
expression.head()

,1000_at,1001_at,1002_f_at,1003_s_at,1004_at,1005_at,1006_at,1007_s_at,1008_f_at,1009_at,...,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at,AFFX-YEL002c/WBP1_at,AFFX-YEL018w/_at,AFFX-YEL021w/URA3_at,AFFX-YEL024w/RIP1_at,AFFX-hum_alu_at
01_a_D_f_2.CEL,9.521934,7.453767,7.045636,7.743690,7.728274,8.081243,6.927978,9.294152,8.888816,8.290944,...,7.065597,6.803698,6.631766,6.981474,7.003319,6.999630,7.005495,6.864895,7.030665,14.394582
01_a_I_f_2.CEL,10.930684,9.455482,9.233291,9.777128,9.612399,9.845444,9.149639,10.580062,11.505571,9.846817,...,9.366604,9.067484,8.945584,9.118070,9.137492,9.067202,9.129742,9.256544,9.171725,15.494106
01_a_M_f_1.CEL,6.852731,5.298974,5.033266,6.040661,5.890083,5.810144,5.098923,6.240855,7.280948,6.124910,...,5.374457,5.165619,5.032617,5.382904,5.509690,5.113946,5.273243,5.321357,5.321636,12.856782
01_c_D_f_1.CEL,7.285181,6.258114,6.119443,6.631768,6.744592,6.589478,6.156638,7.417750,8.663882,8.203275,...,6.332162,6.278864,6.108778,6.341791,6.339638,6.167335,6.170734,6.734936,6.275118,13.683484
01_c_I_f_2.CEL,11.224543,9.800931,9.407753,10.113212,9.871853,10.495533,9.449701,11.084619,11.969619,10.479006,...,9.443910,9.242560,9.272848,9.435175,9.448927,9.395768,9.306998,9.863770,9.404475,15.494106


In [ ]:
genes.head()

,sym,chrom
rownames,,
1000_at,MAPK3,16
1001_at,TIE1,1
1002_f_at,CYP2C19,10
1003_s_at,CXCR5,11
1004_at,CXCR5,11


In [ ]:
samples.head()

,patient,sex,region,lab,chip.version
rownames,,,,,
01_a_D_f_2.CEL,patient_01,female,A.C. cortex,Davis,v2
01_a_I_f_2.CEL,patient_01,female,A.C. cortex,Irvine,v2
01_a_M_f_1.CEL,patient_01,female,A.C. cortex,Michigan,v1
01_c_D_f_1.CEL,patient_01,female,cerebellum,Davis,v1
01_c_I_f_2.CEL,patient_01,female,cerebellum,Irvine,v2


In [37]:
# normalization
genes_Y_crom = genes[genes.chrom == "Y"].index
genes_bac_index = genes[genes.sym.isnull() & genes.chrom.isnull()].index
print(genes_bac_index.shape)
mean_bac_exp = expression[genes_bac_index].mean(axis=1)
print(mean_bac_exp)
expression_norm =  expression - mean_bac_exp.values[:, None]

(868,)
01_a_D_f_2.CEL    7.587471
01_a_I_f_2.CEL    9.588053
01_a_M_f_1.CEL    5.733722
01_c_D_f_1.CEL    6.697715
01_c_I_f_2.CEL    9.879577
                    ...   
10_c_I_f_2.CEL    9.567929
10_c_M_f_1.CEL    5.934693
10_d_D_f_2.CEL    6.905201
10_d_I_f_2.CEL    9.529859
10_d_M_f_2.CEL    6.352657
Length: 84, dtype: float64


In [38]:
# borrowed some code from Vinod
express_samp = pd.concat([expression_norm, samples], axis=1)

express_samp.head()

,1000_at,1001_at,1002_f_at,1003_s_at,1004_at,1005_at,1006_at,1007_s_at,1008_f_at,1009_at,...,AFFX-YEL002c/WBP1_at,AFFX-YEL018w/_at,AFFX-YEL021w/URA3_at,AFFX-YEL024w/RIP1_at,AFFX-hum_alu_at,patient,sex,region,lab,chip.version
01_a_D_f_2.CEL,1.934463,-0.133704,-0.541834,0.156219,0.140803,0.493772,-0.659493,1.706681,1.301345,0.703473,...,-0.587841,-0.581976,-0.722576,-0.556805,6.807111,patient_01,female,A.C. cortex,Davis,v2
01_a_I_f_2.CEL,1.342631,-0.132571,-0.354762,0.189075,0.024346,0.257391,-0.438414,0.992010,1.917518,0.258764,...,-0.520851,-0.458310,-0.331509,-0.416327,5.906053,patient_01,female,A.C. cortex,Irvine,v2
01_a_M_f_1.CEL,1.119009,-0.434748,-0.700456,0.306939,0.156361,0.076422,-0.634799,0.507132,1.547225,0.391188,...,-0.619776,-0.460480,-0.412365,-0.412086,7.123060,patient_01,female,A.C. cortex,Michigan,v1
01_c_D_f_1.CEL,0.587467,-0.439600,-0.578272,-0.065946,0.046877,-0.108237,-0.541076,0.720036,1.966167,1.505561,...,-0.530380,-0.526981,0.037222,-0.422597,6.985769,patient_01,female,cerebellum,Davis,v1
01_c_I_f_2.CEL,1.344966,-0.078646,-0.471824,0.233634,-0.007724,0.615956,-0.429876,1.205042,2.090042,0.599429,...,-0.483809,-0.572580,-0.015807,-0.475102,5.614528,patient_01,female,cerebellum,Irvine,v2


In [6]:
express_samp_UM = express_samp[express_samp.lab == "Michigan"]

p_values = pd.DataFrame()

# for each tissue, permute the 10 outcomes, 
# use the difference-in-means as the test statistic
for tissue in express_samp_UM["region"].unique():
  print(tissue)
  express_samp_UM_new = express_samp_UM[(express_samp_UM.region == tissue)]
  express_samp_UM_new = pd.concat([express_samp_UM_new, pd.DataFrame(columns=['perm'])], axis=1)

  df_male = express_samp_UM_new[express_samp_UM_new.sex == "male"]
  df_female = express_samp_UM_new[express_samp_UM_new.sex == "female"]

  male_mean = df_male.mean()
  female_mean = df_female.mean()

  diff = pd.DataFrame(male_mean - female_mean)
  for comb in combinations(range(10), 5):
    #print(comb)
    express_samp_UM_new.perm = [0] * 10

    express_samp_UM_new.perm.iloc[list(comb)] = 1

    df_male = express_samp_UM_new[express_samp_UM_new.perm == 1]
    df_female = express_samp_UM_new[express_samp_UM_new.perm == 0]

    male_mean = df_male.mean()
    female_mean = df_female.mean()

    diff = pd.concat([diff, male_mean - female_mean], axis=1)
    #print(diff.shape)
  
  # compare columns and get p-values
  diff = diff.abs()
  for j in range(252):
    diff.iloc[:,j+1] = diff.iloc[:,0] <= diff.iloc[:,j+1]
  p_values = pd.concat([p_values, diff.iloc[:,1:].mean(axis=1)], axis=1)

p_values.head()
p_values = p_values.drop(labels=['perm'], axis=0)

A.C. cortex


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3

cerebellum
D.L.P.F. cortex


In [15]:
print(sum(p_values.mean(axis=1) <= 0.05))
print(1/252)
p_values[p_values.mean(axis=1) <= 0.2]
p_values[p_values.min(axis=1) <= 0.008]

5
0.003968253968253968


,0,0,0
1070_at,0.007937,0.976190,0.746032
1246_at,0.896825,0.936508,0.007937
1255_g_at,0.007937,0.896825,0.349206
1579_at,0.698413,0.007937,0.738095
1654_at,0.007937,0.023810,0.460317
1672_f_at,0.865079,0.007937,0.476190
1688_s_at,0.619048,0.007937,0.928571
31454_f_at,0.825397,0.007937,0.595238
33019_at,0.007937,0.571429,0.936508
33386_at,0.952381,0.253968,0.007937


In [42]:
bonferr = multipletests(p_values.iloc[:,0], method='fdr_bh')[0]
sum(bonferr)

0

In [28]:
p_values.columns = ['A.C. cortex', 'cerebellum', 'D.L.P.F. cortex']
p_values_mean = pd.concat([p_values, pd.DataFrame(p_values.mean(axis=1), columns=['mean'])], axis=1)
p_values_mean.head()

,A.C. cortex,cerebellum,D.L.P.F. cortex,mean
1000_at,0.634921,0.222222,0.809524,0.555556
1001_at,0.126984,0.825397,0.865079,0.605820
1002_f_at,0.388889,0.333333,0.603175,0.441799
1003_s_at,0.468254,0.777778,0.944444,0.730159
1004_at,0.079365,0.388889,0.801587,0.423280


In [33]:
p_values_mean[p_values.min(axis=1) <= 0.008].sort_values(by=['mean'], axis=0)

,A.C. cortex,cerebellum,D.L.P.F. cortex,mean
41214_at,0.007937,0.007937,0.007937,0.007937
38355_at,0.007937,0.007937,0.007937,0.007937
38446_at,0.055556,0.007937,0.007937,0.023810
37583_at,0.007937,0.007937,0.071429,0.029101
38585_at,0.015873,0.007937,0.063492,0.029101
969_s_at,0.007937,0.015873,0.150794,0.058201
34477_at,0.015873,0.007937,0.174603,0.066138
1654_at,0.007937,0.023810,0.460317,0.164021
38954_at,0.380952,0.007937,0.206349,0.198413
35001_at,0.111111,0.007937,0.531746,0.216931


In [34]:
genes.loc[p_values_mean[p_values.min(axis=1) <= 0.008].sort_values(by=['mean'], axis=0).index].loc[:, ['sym','chrom']]

,sym,chrom
41214_at,RPS4Y1,Y
38355_at,DDX3Y,Y
38446_at,XIST,X
37583_at,KDM5D,Y
38585_at,NaN,11hbg
969_s_at,NaN,X
34477_at,UTY,Y
1654_at,BRAF,7
38954_at,VIPR2,7
35001_at,KCTD20,6


In [10]:
genes.loc[p_values[p_values.mean(axis=1) <= 0.1].index].loc[:, ['sym','chrom']]

,sym,chrom
32964_at,EMR1,19
34477_at,UTY,Y
35095_r_at,LILRA3,19
37583_at,KDM5D,Y
38355_at,DDX3Y,Y
38446_at,XIST,X
38585_at,NaN,11hbg
41214_at,RPS4Y1,Y
969_s_at,NaN,X
